In [1]:
import pandas as pd
import spacy
from spacy.tokens import DocBin

from sklearn.model_selection import train_test_split

RNG_SEED = 69

In [2]:
# !python -m spacy download en_core_web_sm

In [3]:
# !python -m pip install spacy-transformers

In [4]:
df = pd.read_csv("withoutDuplicates.csv", encoding="utf-8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16597 entries, 0 to 16596
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       16597 non-null  object
 1   genre       16597 non-null  object
 2   urlSpotify  16597 non-null  object
 3   spotifyID   16597 non-null  object
 4   lyrics      16597 non-null  object
dtypes: object(5)
memory usage: 648.4+ KB


In [5]:
df.head()

,title,genre,urlSpotify,spotifyID,lyrics
0,It's Complicated,"[""Pop punk""]",https://play.spotify.com/track/069deq5woglvUnW...,069deq5woglvUnWok4MqTe,"Here we go again, another night of being bumpe..."
1,All Signs Point To Lauderdale,"[""Pop punk""]",https://play.spotify.com/track/15qf0H31MChhqKr...,15qf0H31MChhqKr2zvejla,"I hate this town, it's so washed up\nAnd all m..."
2,Right Back At It Again,"[""Pop punk"",""Post-hardcore""]",https://play.spotify.com/track/0OQmxZOINqWsbKy...,0OQmxZOINqWsbKysfiPSTO,We're coming out swinging\n♪\nStarted out in t...
3,Almost Lover,"[""Adult album alternative""]",https://play.spotify.com/track/71ehTADpxs85ULr...,71ehTADpxs85ULrZgSEKCy,Your fingertips across my skin\nThe palm trees...
4,Blow Away,"[""Adult album alternative"",""Pop music""]",https://play.spotify.com/track/2mzgLMAfvGd7pN6...,2mzgLMAfvGd7pN6JRlNfo3,One of us is gonna be here and\nOne of us is g...


In [6]:
df["genre"] = df["genre"].apply(lambda x: x.lower())
df.head()

,title,genre,urlSpotify,spotifyID,lyrics
0,It's Complicated,"[""pop punk""]",https://play.spotify.com/track/069deq5woglvUnW...,069deq5woglvUnWok4MqTe,"Here we go again, another night of being bumpe..."
1,All Signs Point To Lauderdale,"[""pop punk""]",https://play.spotify.com/track/15qf0H31MChhqKr...,15qf0H31MChhqKr2zvejla,"I hate this town, it's so washed up\nAnd all m..."
2,Right Back At It Again,"[""pop punk"",""post-hardcore""]",https://play.spotify.com/track/0OQmxZOINqWsbKy...,0OQmxZOINqWsbKysfiPSTO,We're coming out swinging\n♪\nStarted out in t...
3,Almost Lover,"[""adult album alternative""]",https://play.spotify.com/track/71ehTADpxs85ULr...,71ehTADpxs85ULrZgSEKCy,Your fingertips across my skin\nThe palm trees...
4,Blow Away,"[""adult album alternative"",""pop music""]",https://play.spotify.com/track/2mzgLMAfvGd7pN6...,2mzgLMAfvGd7pN6JRlNfo3,One of us is gonna be here and\nOne of us is g...


In [7]:
genre_mappings = {
    "pop": "pop",
    "rock": "rock",
    "country": "country",
    "folk": "folk",
    "jazz": "jazz",
    "hip hop": "hip hop",
    "rap": "rap",
    "soul": "soul",
    "blues": "blues",
    "metal": "metal"
}

for genre, replacement in genre_mappings.items():
    df["genre"] = df["genre"].apply(lambda x: replacement if genre in x else x)

df.head()

,title,genre,urlSpotify,spotifyID,lyrics
0,It's Complicated,pop,https://play.spotify.com/track/069deq5woglvUnW...,069deq5woglvUnWok4MqTe,"Here we go again, another night of being bumpe..."
1,All Signs Point To Lauderdale,pop,https://play.spotify.com/track/15qf0H31MChhqKr...,15qf0H31MChhqKr2zvejla,"I hate this town, it's so washed up\nAnd all m..."
2,Right Back At It Again,pop,https://play.spotify.com/track/0OQmxZOINqWsbKy...,0OQmxZOINqWsbKysfiPSTO,We're coming out swinging\n♪\nStarted out in t...
3,Almost Lover,"[""adult album alternative""]",https://play.spotify.com/track/71ehTADpxs85ULr...,71ehTADpxs85ULrZgSEKCy,Your fingertips across my skin\nThe palm trees...
4,Blow Away,pop,https://play.spotify.com/track/2mzgLMAfvGd7pN6...,2mzgLMAfvGd7pN6JRlNfo3,One of us is gonna be here and\nOne of us is g...


In [8]:
df_simplified = df[df["genre"].isin(genre_mappings.values())]
df_simplified.head()

,title,genre,urlSpotify,spotifyID,lyrics
0,It's Complicated,pop,https://play.spotify.com/track/069deq5woglvUnW...,069deq5woglvUnWok4MqTe,"Here we go again, another night of being bumpe..."
1,All Signs Point To Lauderdale,pop,https://play.spotify.com/track/15qf0H31MChhqKr...,15qf0H31MChhqKr2zvejla,"I hate this town, it's so washed up\nAnd all m..."
2,Right Back At It Again,pop,https://play.spotify.com/track/0OQmxZOINqWsbKy...,0OQmxZOINqWsbKysfiPSTO,We're coming out swinging\n♪\nStarted out in t...
4,Blow Away,pop,https://play.spotify.com/track/2mzgLMAfvGd7pN6...,2mzgLMAfvGd7pN6JRlNfo3,One of us is gonna be here and\nOne of us is g...
6,Space Age Love Song,pop,https://play.spotify.com/track/4FMsajq8hKiOmcJ...,4FMsajq8hKiOmcJ7UzOhag,I saw your eyes\nAnd you made me smile\nFor a ...


In [9]:
df_shuffled = df_simplified.sample(frac=1, random_state=RNG_SEED)
# df_shuffled = df_simplified.sample(n=100, random_state=RNG_SEED)

In [10]:
# 60% train, 20% dev, 20% test
df_train_dev, df_test = train_test_split(df_shuffled, test_size=0.2, random_state=RNG_SEED)
df_train, df_dev = train_test_split(df_train_dev, test_size=0.25, random_state=RNG_SEED)

In [11]:
print(f"{len(df_train) = }")
print(f"{len(df_dev) = }")
print(f"{len(df_test) = }")

len(df_train) = 9015
len(df_dev) = 3006
len(df_test) = 3006


In [12]:
df_train.head()

,title,genre,urlSpotify,spotifyID,lyrics
2092,Ma Baker,pop,https://play.spotify.com/track/4SMVEIfMusETeLh...,4SMVEIfMusETeLhl7ptFUh,"Freeze, I'm Ma Baker, put your hands in the ai..."
11335,Ridin' The Storm Out,rock,https://play.spotify.com/track/0SkmxNE5ydxiEN0...,0SkmxNE5ydxiEN0hzJbMac,"Ridin' the storm out, waitin' for the thaw out..."
15676,As Good As I Once Was,country,https://play.spotify.com/track/5KjhrsPiaDtefxG...,5KjhrsPiaDtefxGFfTUmaO,She said I seen you in here before\nI said I b...
2300,Little Drummer Boy,pop,https://play.spotify.com/track/2YTWQAF0A7rzCSp...,2YTWQAF0A7rzCSpfeiHZa5,"Come they told me, pa rum pum-pum-pum\nA new b..."
816,"Do Right Woman, Do Right Man",pop,https://play.spotify.com/track/6uAEJQpAPoYmM6M...,6uAEJQpAPoYmM6Mb0tJN2I,Take me to heart and I'll always love you\nAnd...


In [13]:
nlp = spacy.load("en_core_web_sm")

def save_docs(song_data: pd.DataFrame, output_file: str) -> None:
    db = DocBin()

    def add_doc(lyrics: str, label: str) -> None:
        doc = nlp.make_doc(lyrics)
        doc.cats = {cat: int(cat == label) for cat in genre_mappings.values()}
        db.add(doc)

    for _, row in song_data.iterrows():
        add_doc(row["lyrics"], row["genre"])

    db.to_disk(output_file)


save_docs(df_train, "spacy/train-all.spacy")
save_docs(df_dev, "spacy/dev-all.spacy")
save_docs(df_test, "spacy/test-all.spacy")

In [14]:
# !python -m spacy init fill-config spacy/base_config.cfg spacy/config.cfg

In [15]:
# !python -m spacy train spacy/config.cfg --paths.train spacy/train.spacy  --paths.dev spacy/dev.spacy --output textcat_model

In [16]:
# !python -m spacy evaluate textcat_model/model-best/ --output metrics.json spacy/test.spacy